In [1]:
from dataloader import TrajectoryDataset

dataset = TrajectoryDataset(data_dir='./datasets/eth/train', obs_len=8, pred_len=12, delim='\t')

[Data(x=[10, 4], edge_index=[100, 2]), Data(x=[10, 4], edge_index=[100, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[5, 4], edge_index=[25, 2]), Data(x=[5, 4], edge_index=[25, 2]), Data(x=[4, 4], edge_index=[16, 2]), Data(x=[4, 4], edge_index=[16, 2]), Data(x=[4, 4], edge_index=[16, 2]), Data(x=[4, 4], edge_index=[16, 2]), Data(x=[4, 4], edge_index=[16, 2]), Data(x=[4, 4], edge_index=[16, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[6, 4], edge_index=[36, 2]), Data(x=[9, 4], edge_index=[81, 2]), Data(x=[9, 4], edge_index=[81, 2]), Data(x=[9, 4], edge_index=[81, 2]), Data(x=[9, 4], edge_index=[81, 2]), Data(x=[9, 4], edge_index=[81, 2]), Data(x=[9, 4], edge_index=[81, 2]), Data(x=[11, 4], edge_index=[121, 2]), Data(x=[11, 4], edge_index=[121, 2]), Data(x=[11, 4], edge_index=[121, 2]), Data(x=[8, 4], edge_index=[64, 2]), Data(x=[8, 4], ed